### Analysis part 

#### [Read data](#readdata)
#### [load cases time window](#loadcase)

In [ ]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import dask.array as da

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import patches
from matplotlib import cm

from metpy import calc as mpcalc
from metpy.units import units

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
os.chdir('/w2-data2/willytsai/python_module/')
import SCAI_calc4obj as scai 

In [ ]:
ISCCP_dir = '/w2-data/willytsai/ISCCP/hgg/'
TRMM_dir = '/w2-data2/willytsai/TRMM_3hr/TRMM/'
MERRA2_dir = '/w2-data2/willytsai/MERRA2/'
MERRA2_3d_dir = '/w2-data/willytsai/'
ERA5_dir = '/w2-data/willytsai/ERA5/'
Gridsat_dir = '/w2-data/willytsai/gridsat_BT/remap_0.25deg/'

In [ ]:
def subset_MERRA2(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file 
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        #print(date)
        date_str = datetime.strftime(date,format='%Y%m%d')
        file = sorted(glob('*'+date_str+'*'))[0]
        data = xr.open_dataset(file)
        data = data.sel(lat=slice(lat_cent-lat_inc,lat_cent+lat_inc),
                        lon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
        
        if n == 0: 
            tmp = data; n+=1
        else:
            tmp = xr.concat([tmp,data],'time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def subset_TRMM(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        date_str = datetime.strftime(date,format='%Y%m%d')
        files = sorted(glob('*'+date_str+'*')) # sub-daily
        for file in files:
            data = xr.open_dataset(file)
            data = data.sel(nlat=slice(lat_cent-lat_inc,lat_cent+lat_inc),
                            nlon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
            data = data.assign_coords(time=datetime.strptime(file[5:16],'%Y%m%d.%H'))
        
            if n == 0: 
                tmp = data; n+=1
            else:
                tmp = xr.concat([tmp,data],dim='time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def subset_gridsat(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        date_str = datetime.strftime(date,format='%Y.%m.%d')
        files = sorted(glob('*'+date_str+'*')) # sub-daily
        for file in files:
            data = xr.open_dataset(file)
            data = data.sel(lat=slice(lat_cent-lat_inc,lat_cent+lat_inc)
                            ,lon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
            data = data.assign_coords(time=datetime.strptime(file[11:24],'%Y.%m.%d.%H'))
        
            if n == 0: 
                tmp = data; n+=1
            else:
                tmp = xr.concat([tmp,data],'time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def SCAI_index(prec_sm,bt_sm):

    N_psT = np.zeros(bt_sm.shape[0]) # BT_based
    N_prec = np.zeros(prec_sm.shape[0]) # prec_based
    SCAI_BT = np.zeros(bt_sm.shape[0]) # cldfrac over domain by BT criteria
    SCAI_p = np.zeros(prec_sm.shape[0]) # cldfrac over domain by prec criteria
    
   # N and SCAI indices
    for t in range(bt_sm.shape[0]):
        label_obj, N = scai.labeled_obj(bt_sm[t,:,:].values,cri=240,flag=2) # less than 240 W/m^2
        SCAI_BT[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)
        N_psT[t] = N
    
    for t in range(prec_sm.shape[0]):
        label_obj, N = scai.labeled_obj(prec_sm[t,:,:].values,cri=1,flag=1) # larger than 1mm/day
        SCAI_p[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)    
        N_prec[t] = N

    return (SCAI_BT,SCAI_p,N_psT,N_prec)

<a id='readdata'></a>
### Read data_event from written .dat files

In [ ]:
# load data
os.chdir('/w2-data/willytsai/ISCCP/')

# variable namelist
var_name = np.loadtxt('var_name_2014_2018.dat',dtype='U16')
# event time
var_time = np.loadtxt('time_event_2014_2018.dat',dtype='U13')
time_event = []
for t in var_time:
    tmp = datetime.strptime(t,'%Y-%m-%d-%H')
    time_event.append(tmp)

var_dataset = (np.loadtxt('var_event_2014_2018.dat'))

In [ ]:
os.chdir('/w2-data/willytsai/TQUV_3hr/')
files = glob('*.nc')
lev = xr.open_dataset(files[0]).T.lev # MERRA2 standard level 
del files
os.chdir('/w2-data/willytsai/Cloudfrac_3hr/')
lev_m = np.loadtxt('model_lev.txt')[:,1]# MERRA2 model level, 58 levs [72-15]

In [ ]:
def event_region(lat_event,lon_event,lon_w,lon_e,lat_s,lat_n):
    'select specfic region'
    idx = np.where(np.logical_and(lon_event >= lon_w, lon_event <= lon_e))[0]
    idy = np.where(np.logical_and(lat_event >= lat_s, lat_event <= lat_n))[0]
    idc = np.intersect1d(idx,idy)
    return idc

In [ ]:
lat_event = var_dataset[:,0]
lon_event = var_dataset[:,1]

In [ ]:
os.chdir('/w2-data/willytsai/ISCCP')
T_all = np.load('T_all.mat.npy')
q_all = np.load('q_all.mat.npy')
u_all = np.load('u_all.mat.npy')
v_all = np.load('v_all.mat.npy')
N_all = np.load('N_all.mat.npy')
SCAIbt_all = np.load('SCAIbt_all.mat.npy')
bt_all = np.load('bt_all.mat.npy')
llws_all = np.load('llws_all.mat.npy')
prec_all = np.load('prec_all.mat.npy')
precstd_all = np.load('precstd_all.mat.npy')
cwv_all = np.load('cwv_all.mat.npy')
crh_all = np.load('crh_all.mat.npy')
# dcamt_all = np.load('dcamt_all.mat.npy')
# cuamt_all = np.load('cuamt_all.mat.npy')
# stamt_all = np.load('stamt_all.mat.npy')
omega_all = np.load('omega_all.mat.npy')
rh_all = np.load('rh_all.mat.npy')
cwvstd_all = np.load('cwvstd_all.mat.npy')
SF_all = np.load('SF_all.mat.npy')
cldfrac_all = np.load('cldfrac_all.mat.npy')
CFv_all = np.load('CFv_all.mat.npy')
div_all  = np.load('div_all.mat.npy')
cwvten_dyn_all = np.load('cwvten_dyn_all.mat.npy')
cwvten_phy_all = np.load('cwvten_phy_all.mat.npy')
cwvten_ana_all = np.load('cwvten_ana_all.mat.npy')
cwvten_tot_all = cwvten_dyn_all+cwvten_phy_all+cwvten_ana_all
Abar_all = cldfrac_all/N_all*1600
cape_all = np.load('cape_all.mat.npy')
cin_all = np.load('cin_all.mat.npy')

In [ ]:
def center_case(time_event,lon_event,lat_event,cri_r=300,ratio=0.5):
    'precipitation area is enclosed within a circle R at reference time'
    
    file_path = TRMM_dir+'2014_2018'
    time_cent = time_event
    lat_cent = lat_event
    lon_cent = lon_event
    
    # read preciptation data in the given domain
    data = subset_TRMM(file_path,time_cent,lat_cent,lon_cent,days=0,lat_inc=5,lon_inc=5)
    prec_sm = data.precipitation.squeeze()
    prec_sum = prec_sm.sum()

    # calculate the distance matrix
    x,y = np.meshgrid(prec_sm.nlon,prec_sm.nlat)
    dist = np.sqrt((x-lon_cent)**2+(y-lat_cent)**2)*100 # distance from center [km]
    prec_sm.values[dist > cri_r] = 0
    prec_frac = np.sum(prec_sm)/prec_sum
    
    if prec_frac >= ratio:
        result = 1
    else:
        result = 0

    return result

In [ ]:
def remove_diurnal_1D(data_all):
    'remove diurnal cycle in the time series [N,time], N cases'
    tmp_re = np.copy(data_all)
    NT = data_all.shape[1]
    
    for n in range(data_all.shape[0]):
        tmp = np.fft.fft(data_all[n,:]) # 49
        freqs = np.array(range(NT))   # This gets the left end correct
        reversed_freqs = freqs[::-1]  # Reverse the array 
        reversed_freqs[0:int(NT/2)] = freqs[1:int(NT/2)+1] # Fix the low end of the reversed array
        freqs = reversed_freqs[::-1]  # reverse it back 
        tmp[np.where(freqs==6)] = 0 # remove diurnal 6 cycles for 6 days
        tmp_re[n,:] = np.fft.ifft(tmp) # reconstructed time-series                   
    
    return tmp_re

In [ ]:
def remove_diurnal_2D(data_all):
    'remove diurnal cycle in the time series [N,time,lev], N cases'
    
    tmp_re = np.copy(data_all)
    NT = data_all.shape[1]
    
    for n in range(data_all.shape[0]):
        for k in range(data_all.shape[2]):
            tmp = np.fft.fft(data_all[n,:,k]) # 49
            freqs = np.array(range(NT))   # This gets the left end correct
            reversed_freqs = freqs[::-1]  # Reverse the array 
            reversed_freqs[0:int(NT/2)] = freqs[1:int(NT/2)+1] # Fix the low end of the reversed array
            freqs = reversed_freqs[::-1]  # reverse it back 
            tmp[np.where(freqs==6)] = 0 # remove diurnal 6 cycles for 6 days
            tmp_re[n,:,k] = np.fft.ifft(tmp) # reconstructed time-series                   
    
    return tmp_re

In [ ]:
# select regions
idx = event_region(lat_event,lon_event,40,100,-10,10)
time_event = np.asarray(time_event)[idx]
lat_event = lat_event[idx]
lon_event = lon_event[idx]

In [ ]:
# remove diurnal cycle and marginal cases
T_all = T_all[idx,:,:]; T_all = remove_diurnal_2D(T_all) 
q_all = q_all[idx,:,:]; q_all = remove_diurnal_2D(q_all)
#u_all = u_all[idx,:]
#v_all = v_all[idx,:]
N_all = N_all[idx,:]; N_all = remove_diurnal_1D(N_all)
SCAIbt_all = SCAIbt_all[idx,:]; SCAIbt_all = remove_diurnal_1D(SCAIbt_all)
bt_all = bt_all[idx,:]; bt_all = remove_diurnal_1D(bt_all)
llws_all = llws_all[idx,:]; llws_all = remove_diurnal_1D(llws_all)
prec_all = prec_all[idx,:]; prec_all = remove_diurnal_1D(prec_all)
precstd_all = precstd_all[idx,:]; precstd_all = remove_diurnal_1D(prec_all)
cwv_all = cwv_all[idx,:]; cwv_all = remove_diurnal_1D(cwv_all)
crh_all = crh_all[idx,:]; crh_all = remove_diurnal_1D(crh_all)
# dcamt_all = np.load('dcamt_all.mat.npy')
# cuamt_all = np.load('cuamt_all.mat.npy')
# stamt_all = np.load('stamt_all.mat.npy')
omega_all = omega_all[idx,:,:]; omega_all = remove_diurnal_2D(omega_all)
rh_all = rh_all[idx,:,:]; rh_all = remove_diurnal_2D(rh_all)
cwvstd_all = cwvstd_all[idx,:]; cwvstd_all = remove_diurnal_1D(cwvstd_all)
SF_all = SF_all[idx,:]; SF_all = remove_diurnal_1D(SF_all)
cldfrac_all = cldfrac_all[idx,:]; cldfrac_all = remove_diurnal_1D(cldfrac_all)
CFv_all = CFv_all[idx,:,:]; CFv_all = remove_diurnal_2D(CFv_all)
div_all  = div_all[idx,:,:]; 
cwvten_dyn_all = cwvten_dyn_all[idx,:]; cwvten_dyn_all = remove_diurnal_1D(cwvten_dyn_all)
cwvten_phy_all = cwvten_phy_all[idx,:]; cwvten_phy_all = remove_diurnal_1D(cwvten_phy_all)
cwvten_ana_all = cwvten_ana_all[idx,:]; cwvten_ana_all = remove_diurnal_1D(cwvten_ana_all)
cwvten_tot_all = cwvten_dyn_all+cwvten_phy_all+cwvten_ana_all;
Abar_all = cldfrac_all/N_all*1600
cape_all = cape_all[idx,:]; cape_all = remove_diurnal_1D(cape_all)
cin_all = cin_all[idx,:]; cin_all = remove_diurnal_1D(cin_all)


In [ ]:
lat_bin = np.linspace(-10,10,5)
lon_bin = np.linspace(-175,180,72)
freq_bin = np.zeros((len(lon_bin),len(lat_bin)))
precamt_bin = freq_bin.copy()

fig = plt.figure(figsize=(7,5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))

for i in range(len(lon_bin)):
    for j in range(len(lat_bin)):
        idx = np.where(np.logical_and(lon_event==lon_bin[i],lat_event==lat_bin[j]))[0]
        freq_bin[i,j]+=len(idx)
        if len(idx) > 0:
            precamt_bin[i,j]+=prec_all[idx].sum()

lat_plt = [-10,5,0,5,10]
cs = plt.pcolor(lon_bin,lat_bin[:-1],freq_bin[:,1:].transpose(),cmap='jet',vmin=1)
cs.cmap.set_under(color='k')
cbar = plt.colorbar(fraction=0.02,pad=0.02)
cbar.set_label('event number',fontsize=12)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND
gl = ax.gridlines(draw_labels=True,linestyle='--')
gl.xlabels_top = gl.ylabels_right = gl.ylabels_left = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':13}
ax.set_ylim([-20,20]);ax.set_xlim([40,100])
plt.text(82,-17,'Total event:'+str(prec_all.shape[0]),fontsize=12,color='b')
plt.title('TRMM identified convective event, 2014-2018, IND',fontsize=13)

plt.show()

In [ ]:
# stage--> response and state (precip', cape', cin', crh')
cin_scat = cin_all[np.where(cldfrac_all[:,24]<0.25)[0],:]
cape_scat = cape_all[np.where(cldfrac_all[:,24]<0.25)[0],:]
crh_scat = crh_all[np.where(cldfrac_all[:,24]<0.25)[0],:]

prec_scat = prec_all[np.where(cldfrac_all[:,24]<0.25)[0],:]
cin_regress_scat = np.copy(cin_scat)
cape_regress_scat = np.copy(cape_scat)
crh_regress_scat = np.copy(crh_scat)

for t in range(cin_regress_scat.shape[0]): # samples
    tmp = prec_scat[t,:];tmp2 = -cin_scat[t,:]
    pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
    if len(pp) > 0: # not empty
        reg = [np.polyfit(qq,np.roll(pp,-lag),deg=1)[0] for lag in range(-24,25)]
        cin_regress_scat[t,:] = np.asarray(reg)
    else:
        cin_regress_scat[t,:] = np.nan
        
for t in range(cape_regress_scat.shape[0]): # samples
    tmp = prec_scat[t,:];tmp2 = cape_scat[t,:]
    pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
    if len(pp) > 0: # not empty
        reg = [np.polyfit(qq,np.roll(pp,-lag),deg=1)[0] for lag in range(-24,25)]
        cape_regress_scat[t,:] = np.asarray(reg)
    else:
        cape_regress_scat[t,:] = np.nan

for t in range(crh_regress_scat.shape[0]): # samples
    tmp = prec_scat[t,:];tmp2 = crh_scat[t,:]
    pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
    if len(pp) > 0: # not empty
        reg = [np.polyfit(qq,np.roll(pp,-lag),deg=1)[0] for lag in range(-24,25)]
        crh_regress_scat[t,:] = np.asarray(reg)
    else:
        crh_regress_scat[t,:] = np.nan

In [ ]:
# ## cin_regress_scatmn = np.nanmean(cin_regress_scat,0)
# cape_regress_scatmn = np.nanmean(cape_regress_scat,0)
# crh_regress_scatmn = np.nanmean(crh_regress_scat,0)

# fig,ax = plt.subplots(3,1,figsize=(8,10))

# cf1 = ax[0].plot(range(49),cin_regress_scatmn)
# ax[0].set_xticks(np.linspace(0,48,13));ax[0].set_xticklabels(np.linspace(-72,72,13,dtype=int))
# ax[0].set_xlabel('Lag time [hr]',fontsize=12)

# cf2 = ax[1].plot(range(49),cape_regress_scatmn)
# ax[1].set_xticks(np.linspace(0,48,13));ax[1].set_xticklabels(np.linspace(-72,72,13,dtype=int))
# ax[1].set_xlabel('Lag time [hr]',fontsize=12)

# cf3 = ax[2].plot(range(49),crh_regress_scatmn)
# ax[2].set_xticks(np.linspace(0,48,13));ax[2].set_xticklabels(np.linspace(-72,72,13,dtype=int))
# ax[2].set_xlabel('Lag time [hr]',fontsize=12)

In [ ]:
# CRH and CF for all time
CRH_bin = np.array([0.3,0.5,0.7,1])
CF_bin = np.array([0,0.1,0.25,0.5,1])
#CF_bin = np.linspace(0,1,21)

num_bin = np.zeros((len(CRH_bin)-1,len(CF_bin)-1))
cldfrac_1d = cldfrac_all.ravel()
crh_1d = crh_all.ravel()

for i in range(len(CRH_bin)-1):
    for n in range(len(CF_bin)-1):
        idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[n],cldfrac_1d<CF_bin[n+1]))[0]
        idy = np.where(np.logical_and(crh_1d>=CRH_bin[i],crh_1d<CRH_bin[i+1]))[0]
        idd = np.intersect1d(idx,idy)
        
        num_bin[i,n] = len(idd)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(12,3))
ax[0].bar(range(4),num_bin[0,:],width=0.2)
ax[0].set_title('CRH 0.3-0.5, '+'('+str(round(np.sum(num_bin[0,:])/np.sum(num_bin),3)*100)+'%)')
ax[1].bar(range(4),num_bin[1,:],width=0.2)
ax[1].set_title('CRH 0.5-0.7, '+'('+str(round(np.sum(num_bin[1,:])/np.sum(num_bin),3)*100)+'%)')
ax[2].bar(range(4),num_bin[2,:],width=0.2)
ax[2].set_title('CRH 0.7-1, '+'('+str(round(np.sum(num_bin[2,:])/np.sum(num_bin),3)*100)+'%)')

In [ ]:
# boxplot for all
CF_bin = np.array([0,0.1,0.25,0.5,1])

cldfrac_1d = cldfrac_all.ravel()
crh_1d = crh_all.ravel()

crh_comall = []
for n in range(len(CF_bin)-1):
    idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[n],cldfrac_1d<CF_bin[n+1]))[0]
    tmp = crh_1d[idx]
    tmp2 = tmp[tmp>0]
    crh_comall.append(tmp2)


In [ ]:
# boxplot for reference time
CF_bin = np.array([0,0.1,0.25,0.5,1])

cldfrac_1d = cldfrac_all[:,24].ravel()
crh_1dref = crh_all[:,24].ravel()
crh_1dp = crh_all[:,40].ravel()
crh_1db = crh_all[:,16].ravel()

crh_comref = []
crh_com12p = []
crh_com12b = []
for n in range(len(CF_bin)-1):
    idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[n],cldfrac_1d<CF_bin[n+1]))[0]
    tmp = crh_1d[idx];tmp2 = tmp[tmp>0]
    crh_comref.append(tmp2)
    
    tmp = crh_1dp[idx];tmp2 = tmp[tmp>0]
    crh_com12p.append(tmp2)
    
    tmp = crh_1db[idx];tmp2 = tmp[tmp>0]
    crh_com12b.append(tmp2)


In [ ]:
for n in range(len(crh_comref)):
    print(len(crh_comref[n]))

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(14,5))
ax[0].boxplot(crh_comall, 0, 'gd',labels=['0 - 0.1','0.1 - 0.25','0.25 - 0.5','0.5 - 1'])
ax[0].tick_params(labelsize=12)
ax[0].set_ylim([0.2,1])

ax[1].boxplot(crh_comref, 0, 'gd',labels=['0 - 0.1','0.1 - 0.25','0.25 - 0.5','0.5 - 1'])
ax[1].tick_params(labelsize=12)
ax[1].set_ylim([0.2,1])

plt.show()

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# auto-correlation of precipitation binned by reference CF & CRH
#CRH_bin = np.linspace(0.6,0.85,6)
CRH_bin = np.array([0,1])
#CF_bin = np.linspace(0,1,11)
CF_bin = np.array([0,0.25,0.5,1])

ef_time = np.zeros((len(CRH_bin)-1,len(CF_bin)-1))
ef_array = []
ef_num = np.copy(ef_time)
cldfrac_1d = cldfrac_all[:,24].ravel()
crh_1d = crh_all[:,24].ravel()

for i in range(len(CRH_bin)-1):
    for n in range(len(CF_bin)-1):
        idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[n],cldfrac_1d<CF_bin[n+1]))[0]
        idy = np.where(np.logical_and(crh_1d>=CRH_bin[i],crh_1d<CRH_bin[i+1]))[0]
        idd = np.intersect1d(idx,idy)
                
        tmp2=[]
        for k in idd:
            if len(idd)>0:
                pp = crh_all[k,:]
                r = [np.corrcoef(pp,np.roll(pp,-lag))[0,1] for lag in range(-24,25)] 
                ratio = r/r[24]
                tmp = find_nearest(ratio, 1/np.e)
                if tmp < 24:
                    tmp2.append(24+(24-tmp))
                else:
                    tmp2.append(tmp)
        ef_time[i,n]=np.nanmean(np.asarray(tmp2))
        ef_array.append(tmp2)
        ef_num[i,n]=len(tmp2)

In [ ]:
for n in range(3):
    plt.plot(CRH_bin[:-1],ef_time[:,n].transpose(),'s')
plt.xlim([0.55,0.9])#;plt.ylim([0,1])
plt.xlabel('CRH',fontsize=12);plt.ylabel('e-folding time',fontsize=12)
plt.legend(['0-0.25','0.25-0.5','0.5-1'])

In [ ]:
# CRH and CF for all time joint histogram
CRH_bin = np.linspace(0.3,1,15)
CF_bin = np.linspace(0,1,21)

num_bin = np.zeros((len(CRH_bin)-1,len(CF_bin)-1))
cldfrac_1d = cldfrac_all.ravel()
crh_1d = crh_all.ravel()

for i in range(len(CRH_bin)-1):
    for n in range(len(CF_bin)-1):
        idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[n],cldfrac_1d<CF_bin[n+1]))[0]
        idy = np.where(np.logical_and(crh_1d>=CRH_bin[i],crh_1d<CRH_bin[i+1]))[0]
        idd = np.intersect1d(idx,idy)
        
        num_bin[i,n] = len(idd)

In [ ]:
levs = np.linspace(1e-3,1e-1,13)
plt.contourf(CRH_bin[:-1],CF_bin[:-1],(num_bin/np.sum(num_bin)).transpose()
            ,levels=levs,cmap='jet')
plt.colorbar()
plt.xlim([0.2,1]);plt.ylim([0,1])
plt.xlabel('CRH',fontsize=12);plt.ylabel('CF',fontsize=12)

In [ ]:
##### SCAI distribution by CF

CF_bin = np.array([0,0.1,0.25,0.5,1])
cldfrac_1d = cldfrac_all[:,24].ravel()
scai_1d = SCAIbt_all[:,24].ravel()

scai_bin = []

for i in range(len(CF_bin)-1):
    idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[i],cldfrac_1d<CF_bin[i+1]))[0]
    scai_bin.append(scai_1d[idx])

fig,ax = plt.subplots(1,4,figsize=(11,3))    
ax[0].hist(scai_bin[0],bins=11,range=(0,25),density=True,histtype='step',linewidth=1.2)
ax[0].set_title('CF_ref: 0-0.1');ax[0].set_ylim([0,0.16])
ax[0].text(18,0.14,str(len(scai_bin[0])),fontsize=12,color='grey')
ax[0].set_xlabel('SCAI')

ax[1].hist(scai_bin[1],bins=11,range=(0,25),density=True,histtype='step',linewidth=1.2)
ax[1].set_title('CF_ref: 0.1-0.25');ax[1].set_ylim([0,0.16])
ax[1].text(18,0.14,str(len(scai_bin[1])),fontsize=12,color='grey')
ax[1].set_xlabel('SCAI')

ax[2].hist(scai_bin[2],bins=11,range=(0,25),density=True,histtype='step',linewidth=1.2)
ax[2].set_title('CF_ref: 0.25-0.5');ax[2].set_ylim([0,0.16])
ax[2].text(18,0.14,str(len(scai_bin[2])),fontsize=12,color='grey')
ax[2].set_xlabel('SCAI')

ax[3].hist(scai_bin[3],bins=11,range=(0,25),density=True,histtype='step',linewidth=1.2)
ax[3].set_title('CF_ref: 0.5-1');ax[3].set_ylim([0,0.16])
ax[3].text(18,0.14,str(len(scai_bin[3])),fontsize=12,color='grey')
ax[3].set_xlabel('SCAI')

plt.tight_layout()


In [ ]:
# N distribution by CF

CF_bin = np.array([0,0.1,0.25,0.5,1])
cldfrac_1d = cldfrac_all[:,24].ravel()
N_1d = N_all[:,24].ravel()

N_bin = []

for i in range(len(CF_bin)-1):
    idx = np.where(np.logical_and(cldfrac_1d>=CF_bin[i],cldfrac_1d<CF_bin[i+1]))[0]
    N_bin.append(N_1d[idx])

fig,ax = plt.subplots(1,4,figsize=(10,3))    
ax[0].hist(N_bin[0],bins=11,range=(0,50),density=True,histtype='step')
ax[0].set_title(len(N_bin[0]))
ax[1].hist(N_bin[1],bins=11,range=(0,50),density=True,histtype='step')
ax[1].set_title(len(N_bin[1]))
ax[2].hist(N_bin[2],bins=11,range=(0,50),density=True,histtype='step')
ax[2].set_title(len(N_bin[2]))
ax[3].hist(N_bin[3],bins=11,range=(0,50),density=True,histtype='step')
ax[3].set_title(len(N_bin[3]))
plt.tight_layout()



In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'same') / w

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) # mm/day
preccom_bin = np.zeros((len(cldfrac_bin)-1,49))
precse_bin = np.copy(preccom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    preccom_bin[n,:] = np.nanmean(prec_all[idx,:],axis=0)
    precse_bin[n,:] = np.nanstd(prec_all[idx,:],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[0,:],preccom_bin[0,:]+2*precse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[0,:],preccom_bin[0,:]-2*precse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),preccom_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[1,:],preccom_bin[1,:]+2*precse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[1,:],preccom_bin[1,:]-2*precse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),preccom_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[2,:],preccom_bin[2,:]+2*precse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[2,:],preccom_bin[2,:]-2*precse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),preccom_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[3,:],preccom_bin[3,:]+2*precse_bin[3,:],color='r'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),preccom_bin[3,:],preccom_bin[3,:]-2*precse_bin[3,:],color='r'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),preccom_bin[3,:],'r')

plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('Precipitation [mm/hr]',fontsize=12);#plt.ylim([0.6,0.85])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) # mm/day
cwvcom_bin = np.zeros((len(cldfrac_bin)-1,49))
cwvse_bin = np.copy(cwvcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwvcom_bin[n,:] = np.nanmean(cwv_all[idx,:],axis=0)
    cwvse_bin[n,:] = np.nanstd(cwv_all[idx,:],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[0,:],cwvcom_bin[0,:]+2*cwvse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[0,:],cwvcom_bin[0,:]-2*cwvse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvcom_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[1,:],cwvcom_bin[1,:]+2*cwvse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[1,:],cwvcom_bin[1,:]-2*cwvse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvcom_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[2,:],cwvcom_bin[2,:]+2*cwvse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[2,:],cwvcom_bin[2,:]-2*cwvse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvcom_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[3,:],cwvcom_bin[3,:]+2*cwvse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvcom_bin[3,:],cwvcom_bin[3,:]-2*cwvse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvcom_bin[3,:],'red')

plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('CWV [mm]',fontsize=12);#plt.ylim([0.6,0.85])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) # mm/day
cfcom_bin = np.zeros((len(cldfrac_bin)-1,49))
cfse_bin = np.copy(cfcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cfcom_bin[n,:] = np.nanmean(cldfrac_all[idx,:],axis=0)
    cfse_bin[n,:] = np.nanstd(cldfrac_all[idx,:],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[0,:],cfcom_bin[0,:]+2*cfse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[0,:],cfcom_bin[0,:]-2*cfse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cfcom_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[1,:],cfcom_bin[1,:]+2*cfse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[1,:],cfcom_bin[1,:]-2*cfse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cfcom_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[2,:],cfcom_bin[2,:]+2*cfse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[2,:],cfcom_bin[2,:]-2*cfse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cfcom_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[3,:],cfcom_bin[3,:]+2*cfse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cfcom_bin[3,:],cfcom_bin[3,:]-2*cfse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cfcom_bin[3,:],'red')

plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('CF',fontsize=12);#plt.ylim([0.6,0.85])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) # mm/day
crhcom_bin = np.zeros((len(cldfrac_bin)-1,49))
crhse_bin = np.copy(crhcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    crhcom_bin[n,:] = np.nanmean(crh_all[idx,:],axis=0)
    crhse_bin[n,:] = np.nanstd(crh_all[idx,:],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)
    
fig = plt.figure(figsize=(8,5))
levs = np.linspace(0.6,0.8,21)
cs = plt.contourf(np.linspace(-72,72,49,dtype=int),cldfrac_bin[:-1],crhcom_bin,levels=levs,
            cmap='jet',extend='both')
cs.cmap.set_under(color='k')
#cs.set_clim(-6,6)
cbar = plt.colorbar(cs,orientation='horizontal',pad=0.15,shrink=0.8)
cbar.set_label('CRH [%]')
plt.ylabel('cloud fraction')
plt.xlabel('Time [hr]')
plt.title('compoiste cwv anomaly ranked by precipi max, IND')
plt.grid(axis='x',linestyle='--')

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[0,:],crhcom_bin[0,:]+2*crhse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[0,:],crhcom_bin[0,:]-2*crhse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),crhcom_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[1,:],crhcom_bin[1,:]+2*crhse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[1,:],crhcom_bin[1,:]-2*crhse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),crhcom_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[2,:],crhcom_bin[2,:]+2*crhse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[2,:],crhcom_bin[2,:]-2*crhse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),crhcom_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[3,:],crhcom_bin[3,:]+2*crhse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),crhcom_bin[3,:],crhcom_bin[3,:]-2*crhse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),crhcom_bin[3,:],'red')

plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('CRH',fontsize=12);plt.ylim([0.6,0.85])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
llws_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
llwse_bin = np.copy(llws_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    llws_bin[n,:] = np.nanmean(llws_all[idx,:],axis=0)
    llwse_bin[n,:] = np.nanstd(llws_all[idx,:],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[0,:],llws_bin[0,:]+2*llwse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[0,:],llws_bin[0,:]-2*llwse_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),llws_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[1,:],llws_bin[1,:]+2*llwse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[1,:],llws_bin[1,:]-2*llwse_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),llws_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[2,:],llws_bin[2,:]+2*llwse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[2,:],llws_bin[2,:]-2*llwse_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),llws_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[3,:],llws_bin[3,:]+2*llwse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),llws_bin[3,:],llws_bin[3,:]-2*llwse_bin[3,:],color='red'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),llws_bin[3,:],'red')

plt.legend(['0 < CF$_{ref}$ < 0.1','0 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('LLWS [m/s]',fontsize=12);plt.ylim([5,7])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cape_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
capestd_bin = np.zeros((len(cldfrac_bin)-1,49))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cape_bin[n,:] = np.nanmean(cape_all[idx,:],axis=0)
    capestd_bin[n,:] = np.nanstd(cape_all[idx,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
          ,fontsize=9)
plt.ylabel('CAPE [J/kg]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
import scipy.stats as sp

In [ ]:
pp = sp.linregress(cldfrac_all[:,24],prec_all[:,24])
pp

In [ ]:
x = np.linspace(0,1,100)
line_fit = x*pp[0]+pp[1]

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(cldfrac_all[:,24],prec_all[:,24],edgecolor='k',facecolor='royalblue')
plt.plot(x,line_fit,'r')
plt.xlabel('Cloud fraction',fontsize=13)
plt.ylabel('Precipitation [mm/hr]',fontsize=13)
plt.text(0,2.7,'R-value='+str(round(pp[2],3)),fontsize=12,color='g')
plt.text(0,2.9,'N='+str(prec_all.shape[0]),fontsize=12,color='g')
plt.tick_params(labelsize=11)
plt.ylim([0,3.2]);plt.xlim([-0.1,1]);plt.grid(linestyle=':')
plt.title('Reference time/ Prec Max')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cin_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cin_bin[n,:] = np.nanmean(cin_all[idx,:],axis=0)
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),-cin_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),-cin_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),-cin_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),-cin_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 0.75']
          ,fontsize=9)
plt.ylabel('CIN [J/kg]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
scai_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    scai_bin[n,:] = np.nanmean(SCAIbt_all[idx,:],axis=0)
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),scai_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),scai_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),scai_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),scai_bin[3,:],color='r')
plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 0.75']
          ,fontsize=9)
plt.ylabel('SCAI')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cwvstd_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
cwvstde_bin = np.copy(cwvstd_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwvstd_bin[n,:] = np.nanmean(cwvstd_all[idx,:]/cwv_all[idx,:],axis=0)
    cwvstde_bin[n,:] = np.nanstd(cwvstd_all[idx,:]/cwv_all[idx,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)

In [ ]:
fig=plt.figure(figsize=(6,4))

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[0,:],cwvstd_bin[0,:]+2*cwvstde_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[0,:],cwvstd_bin[0,:]-2*cwvstde_bin[0,:],color='k'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvstd_bin[0,:],'k')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[1,:],cwvstd_bin[1,:]+2*cwvstde_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[1,:],cwvstd_bin[1,:]-2*cwvstde_bin[1,:],color='b'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvstd_bin[1,:],'b')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[2,:],cwvstd_bin[2,:]+2*cwvstde_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[2,:],cwvstd_bin[2,:]-2*cwvstde_bin[2,:],color='orange'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvstd_bin[2,:],'orange')

plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[3,:],cwvstd_bin[3,:]+2*cwvstde_bin[3,:],color='r'
                 ,alpha=0.1,edgecolor=None)
plt.fill_between(np.linspace(-72,72,49,dtype=int),cwvstd_bin[3,:],cwvstd_bin[3,:]-2*cwvstde_bin[3,:],color='r'
                 ,alpha=0.1,edgecolor=None)
plt.plot(np.linspace(-72,72,49,dtype=int),cwvstd_bin[3,:],'r')

plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','CF$_{ref}$ > 0.5']
          ,fontsize=9)
plt.ylabel('Coefficient of variance, CWV',fontsize=12);plt.ylim([0.05,0.2])
plt.xlabel('Lag time [hr]',fontsize=12);plt.xlim([-75,75])
plt.grid(axis='x',linestyle='--')

In [ ]:
# ABL (below 900hpa) RH evolution, temperature evolution
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
rhabl_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    rhabl_bin[n,:] = np.nanmean(rh_all[idx,:,:5],axis=(0,2))
    num_bin[n] = len(idx)

fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),rhabl_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),rhabl_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),rhabl_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),rhabl_bin[3,:],color='r')
plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','0.5 < CF$_{ref}$ < 1']
          ,fontsize=9)
plt.ylabel('RH [%]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
plt.title('RH ABL evolution, 10deg box, IND')

In [ ]:
# Low trooposphere (900hpa-700hPa) RH evolution, temperature evolution
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) 
rhlt_bin = np.zeros((len(cldfrac_bin)-1,49)) 
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    rhlt_bin[n,:] = np.nanmean(rh_all[idx,:,4:13],axis=(0,2))
    num_bin[n] = len(idx)

fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[3,:],color='r')
plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','0.5 < CF$_{ref}$ < 1']
          ,fontsize=9)
plt.ylabel('RH [%]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
plt.title('RH LT evolution, 10deg box, IND')

In [ ]:
# Mid trooposphere (900hpa-700hPa) RH evolution, temperature evolution
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) 
rhlt_bin = np.zeros((len(cldfrac_bin)-1,49)) 
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    rhlt_bin[n,:] = np.nanmean(rh_all[idx,:,12:19],axis=(0,2))
    num_bin[n] = len(idx)

fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),rhlt_bin[3,:],color='r')
plt.legend(['0 < CF$_{ref}$ < 0.1','0.1 < CF$_{ref}$ < 0.25','0.25 < CF$_{ref}$ < 0.5','0.5 < CF$_{ref}$ < 1']
          ,fontsize=9)
plt.ylabel('RH [%]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
plt.title('RH MT evolution, 10deg box, IND')
plt.ylim([40,90])

### vertical structure 

In [ ]:
# define climatology of vertical structure over tropics
rh_clim = np.tile(np.nanmean(rh_all,axis=(0,1)),[49,1])
qv_clim = np.tile(np.nanmean(q_all,axis=(0,1)),[49,1])

In [ ]:
def vertmean_time(data_all):
    'input 3D array [case,time,vert], return 2D array with time-averaged values'
    N = data_all.shape[0] # number of case
    T = data_all.shape[1] # time points
    data_anomaly = data_all.copy()
    
    for i in range(N):
        v_mean = data_all[i,:,:].mean(axis=0)
        data_anomaly[i,:,:] = data_all[i,:,:]-np.tile(v_mean,[T,1])
        
    data_amn = np.nanmean(data_anomaly,0)
    return data_amn

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) 
div_bin = np.zeros((len(cldfrac_bin)-1,49,42)) 
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    
    div_bin[n,:,:] = np.nanmean(rh_all[idx,:,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) 
Ta_bin = np.zeros((len(cldfrac_bin)-1,49,42)) 
Tmn_bin = np.copy(Ta_bin)
Tmn_alltime = np.zeros((len(cldfrac_bin)-1,42))
Tstd_bin = np.copy(Tmn_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    
    Tmn_bin[n,:,:] = np.nanmean(T_all[idx,:,:],axis=0)
    Tstd_bin[n,:,:] = np.nanstd(T_all[idx,:,:],axis=0)
    Ta_bin[n,:,:] = vertmean_time(T_all[idx,:,:]) # mean anomaly of var
    Tmn_alltime[n,:] = np.nanmean(T_all[idx,:,:],axis=(0,1))
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) 
div_bin = np.zeros((len(cldfrac_bin)-1,49,42)) 
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    
    div_bin[n,:,:] = np.nanmean(div_all[idx,:,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) 
rha_bin = np.zeros((len(cldfrac_bin)-1,49,42)) 
rhmn_bin = np.copy(rha_bin)
rhmn_alltime = np.zeros((len(cldfrac_bin)-1,42))
rhstd_bin = np.copy(rhmn_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    
    rhmn_bin[n,:,:] = np.nanmean(rh_all[idx,:,:],axis=0)
    rhstd_bin[n,:,:] = np.nanstd(rh_all[idx,:,:],axis=0)
    rha_bin[n,:,:] = vertmean_time(rh_all[idx,:,:]) # mean anomaly of var
    rhmn_alltime[n,:] = np.nanmean(rh_all[idx,:,:],axis=(0,1))
    num_bin[n] = len(idx)

In [ ]:
fig,ax = plt.subplots(4,1,figsize=(6,12))

cs1 = ax[0].contourf(range(-24,25),lev,(rha_bin[0,:,:]).transpose(),
                    cmap='PiYG',vmin=-10,vmax=10)
ax[0].set_ylim([1000,100]);
cs2 = ax[1].contourf(range(-24,25),lev,(rha_bin[1,:,:]).transpose(),
                    cmap='PiYG',vmin=-10,vmax=10)
ax[1].set_ylim([1000,100]);
cs1 = ax[2].contourf(range(-24,25),lev,(rha_bin[2,:,:]).transpose(),
                    cmap='PiYG',vmin=-10,vmax=10)
ax[2].set_ylim([1000,100]);
cs1 = ax[3].contourf(range(-24,25),lev,(rha_bin[3,:,:]).transpose(),
                    cmap='PiYG',vmin=-10,vmax=10)
ax[3].set_ylim([1000,100]);

#cbar = plt.colorbar(cs1,orientation='horizontal',shrink=0.5,pad=0.2)

In [ ]:
fig,ax = plt.subplots(4,1,figsize=(6,12))
clevs = np.linspace(-0.5,0.5,21)

cs1 = ax[0].contourf(range(-24,25),lev,(Ta_bin[0,:,:]).transpose(),levels=clevs,
                    cmap='PiYG')
ax[0].set_ylim([1000,100]);
cs2 = ax[1].contourf(range(-24,25),lev,(Ta_bin[1,:,:]).transpose(),levels=clevs,
                    cmap='PiYG')
ax[1].set_ylim([1000,100]);
cs1 = ax[2].contourf(range(-24,25),lev,(Ta_bin[2,:,:]).transpose(),levels=clevs,
                    cmap='PiYG')
ax[2].set_ylim([1000,100]);
cs1 = ax[3].contourf(range(-24,25),lev,(Ta_bin[3,:,:]).transpose(),levels=clevs,
                    cmap='PiYG')
ax[3].set_ylim([1000,100]);

#cbar = plt.colorbar(cs1,orientation='horizontal',shrink=0.5,pad=0.2)

In [ ]:
plt.plot(Ta_bin[3,24,:30],lev[:30])
plt.ylim([1000,100])

In [ ]:
fig,ax = plt.subplots(4,1,figsize=(6,12))
clevs = np.linspace(-1,1,21)

cs1 = ax[0].contourf(range(-24,25),lev,(div_bin[0,:,:]*1e5).transpose(),levels=clevs,
                    cmap='PiYG')
ax[0].set_ylim([1000,100]);
cs2 = ax[1].contourf(range(-24,25),lev,(div_bin[1,:,:]*1e5).transpose(),levels=clevs,
                    cmap='PiYG')
ax[1].set_ylim([1000,100]);
cs3 = ax[2].contourf(range(-24,25),lev,(div_bin[2,:,:]*1e5).transpose(),levels=clevs,
                    cmap='PiYG')
ax[2].set_ylim([1000,100]);
cs4 = ax[3].contourf(range(-24,25),lev,(div_bin[3,:,:]*1e5).transpose(),levels=clevs,
                    cmap='PiYG')
ax[3].set_ylim([1000,100]);

#cbar = plt.colorbar(cs4,orientation='horizontal',shrink=0.8,pad=0.2)

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(14,3))

ax[0].plot(rhmn_bin[0,24,:],lev,'k')
ax[0].plot(rhmn_bin[0,16,:],lev,'--b')
ax[0].plot(rhmn_bin[0,32,:],lev,'orange')
ax[0].plot(rhmn_bin[0,40,:],lev,'r')
ax[0].set_ylim([1000,100]);ax[0].set_xlim([40,90])
ax[0].set_xlabel('RH [%]')
ax[0].legend(['Ref','-1d','+1d','+2d']);ax[0].grid(linestyle=':')
ax[0].set_title('CF_ref: 0-0.1')

ax[1].plot(rhmn_bin[1,24,:],lev,'k')
ax[1].plot(rhmn_bin[1,16,:],lev,'--b')
ax[1].plot(rhmn_bin[1,32,:],lev,'orange')
ax[1].plot(rhmn_bin[1,40,:],lev,'r')
ax[1].set_ylim([1000,100]);ax[1].set_xlim([40,90])
ax[1].set_xlabel('RH [%]');ax[1].grid(linestyle=':')
ax[1].set_title('CF_ref: 0.1-0.25')

ax[2].plot(rhmn_bin[2,24,:],lev,'k')
ax[2].plot(rhmn_bin[2,16,:],lev,'--b')
ax[2].plot(rhmn_bin[2,32,:],lev,'orange')
ax[2].plot(rhmn_bin[2,40,:],lev,'r')
ax[2].set_ylim([1000,100]);ax[2].set_xlim([40,90])
ax[2].set_xlabel('RH [%]');ax[2].grid(linestyle=':')
ax[2].set_title('CF_ref: 0.25-0.5')

ax[3].plot(rhmn_bin[3,24,:],lev,'k')
ax[3].plot(rhmn_bin[3,16,:],lev,'--b')
ax[3].plot(rhmn_bin[3,32,:],lev,'orange')
ax[3].plot(rhmn_bin[3,40,:],lev,'r')
ax[3].set_ylim([1000,100]);ax[3].set_xlim([40,90])
ax[3].set_xlabel('RH [%]');ax[3].grid(linestyle=':')
ax[3].set_title('CF_ref: 0.5-1.0')

#plt.tight_layout()

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,10]) 
qva_bin = np.zeros((len(cldfrac_bin)-1,49,42)) 
qvmn_bin = np.copy(qva_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    
    qvmn_bin[n,:,:] = np.nanmean(q_all[idx,:,:],axis=0)
    qva_bin[n,:,:] = vertmean_time(q_all[idx,:,:])
    num_bin[n] = len(idx)

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(14,3))

ax[0].plot(qvmn_bin[0,24,:]*1e3,lev,'k')
ax[0].plot(qvmn_bin[0,16,:]*1e3,lev,'--b')
ax[0].plot(qvmn_bin[0,32,:]*1e3,lev,'orange')
ax[0].plot(qvmn_bin[0,40,:]*1e3,lev,'r')
ax[0].set_ylim([1000,100]);ax[0].set_xlim([0,18])
ax[0].set_xlabel('qv [g/kg]')
ax[0].legend(['Ref','-1d','+1d','+2d']);ax[0].grid(linestyle=':')
ax[0].set_title('CF_ref: 0-0.1')

ax[1].plot(qvmn_bin[1,24,:]*1e3,lev,'k')
ax[1].plot(qvmn_bin[1,16,:]*1e3,lev,'--b')
ax[1].plot(qvmn_bin[1,32,:]*1e3,lev,'orange')
ax[1].plot(qvmn_bin[1,40,:]*1e3,lev,'r')
ax[1].set_ylim([1000,100]);ax[1].set_xlim([0,18])
ax[1].set_xlabel('qv [g/kg]');ax[1].grid(linestyle=':')
ax[1].set_title('CF_ref: 0.1-0.25')

ax[2].plot(qvmn_bin[2,24,:]*1e3,lev,'k')
ax[2].plot(qvmn_bin[2,16,:]*1e3,lev,'--b')
ax[2].plot(qvmn_bin[2,32,:]*1e3,lev,'orange')
ax[2].plot(qvmn_bin[2,40,:]*1e3,lev,'r')
ax[2].set_ylim([1000,100]);ax[2].set_xlim([0,18])
ax[2].set_xlabel('qv [g/kg]');ax[2].grid(linestyle=':')
ax[2].set_title('CF_ref: 0.25-0.5')

ax[3].plot(qvmn_bin[3,24,:]*1e3,lev,'k')
ax[3].plot(qvmn_bin[3,16,:]*1e3,lev,'--b')
ax[3].plot(qvmn_bin[3,32,:]*1e3,lev,'orange')
ax[3].plot(qvmn_bin[3,40,:]*1e3,lev,'r')
ax[3].set_ylim([1000,100]);ax[3].set_xlim([0,18])
ax[3].set_xlabel('qv [g/kg]');ax[3].grid(linestyle=':')
ax[3].set_title('CF_ref: 0.5-1.0')

### vars regressed by precip

In [ ]:
os.chdir('/w2-data/willytsai/TQUV_3hr/')
files = glob('*.nc')
data = xr.open_dataset(files[0])
lev = data.lev

In [ ]:
%%time
div_scat = div_all[np.where(cldfrac_all[:,24]<0.25)[0],:,:]
prec_scat = prec_all[np.where(cldfrac_all[:,24]<0.25)[0],:]
div_regress_scat = np.copy(div_scat)

for t in range(div_regress_scat.shape[0]): # samples
    for k in range(div_regress_scat.shape[2]): # vertical dims
        tmp = prec_scat[t,:];tmp2 = div_scat[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            div_regress_scat[t,:,k] = np.asarray(reg)
        else:
            div_regress_scat[t,:,k] = np.nan
#---------
div_mid = div_all[np.where(np.logical_and(cldfrac_all[:,24]>0.25,cldfrac_all[:,24]<0.5))[0],:,:]
prec_mid = prec_all[np.where(np.logical_and(cldfrac_all[:,24]>0.25,cldfrac_all[:,24]<0.5))[0],:]
div_regress_mid = np.copy(div_mid)
for t in range(div_regress_mid.shape[0]): # samples
    for k in range(div_regress_mid.shape[2]): # vertical dims
        tmp = prec_mid[t,:];tmp2 = div_mid[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            div_regress_mid[t,:,k] = np.asarray(reg)
        else:
            div_regress_mid[t,:,k] = np.nan
#----------
# lag-height regressed div and T based on precip
div_org = div_all[np.where(cldfrac_all[:,24]>0.5)[0],:,:]
prec_org = prec_all[np.where(cldfrac_all[:,24]>0.5)[0],:]
div_regress_org = np.copy(div_org)

for t in range(div_regress_org.shape[0]): # samples
    for k in range(div_regress_org.shape[2]): # vertical dims
        tmp = prec_org[t,:];tmp2 = div_org[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            div_regress_org[t,:,k] = np.asarray(reg)
        else:
            div_regress_org[t,:,k] = np.nan

In [ ]:
# composite significance 
stderr_scat = np.nanstd(div_regress_scat,0)/div_regress_scat.shape[0]
stderr_mid = np.nanstd(div_regress_mid,0)/div_regress_mid.shape[0]
stderr_org = np.nanstd(div_regress_org,0)/div_regress_org.shape[0]

div_regress_scatmn = np.nanmean(div_regress_scat,0)
#div_regress_scatmn[abs(div_regress_scatmn)<2*stderr_scat] = np.nan

div_regress_midmn = np.nanmean(div_regress_mid,0)
#div_regress_midmn[abs(div_regress_midmn)<2*stderr_mid] = np.nan

div_regress_orgmn = np.nanmean(div_regress_org,0)
#div_regress_orgmn[abs(div_regress_orgmn)<2*stderr_org] = np.nan

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(8,10))

levs = np.linspace(-5,5,21)
cf1 = ax[0].contourf(range(49),lev,1e6*div_regress_scatmn.transpose(),levels=levs
                   ,cmap='PiYG',extend='both')
cs1 = ax[0].contour(range(49),lev,1e6*div_regress_scatmn.transpose()
            ,levels=[-5,-3,-1,1,3,5],colors=['k'], linewidths=0.6)
cs1.clabel(fmt='%2d')
#cbar1 = plt.colorbar(cf1,pad=0.01);cbar1.set_label('Reg [g/kg]/[mm/hr]')
ax[0].set_ylim([1000,200]);ax[0].grid(linestyle='--')
ax[0].set_xticks(np.linspace(0,48,13));ax[0].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[0].set_xlabel('Lag time [hr]',fontsize=12)
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)

cf2 = ax[1].contourf(range(49),lev,1e6*div_regress_midmn.transpose(),levels=levs
                   ,cmap='PiYG',extend='both')
cs2 = ax[1].contour(range(49),lev,1e6*div_regress_midmn.transpose()
            ,levels=[-5,-3,-1,1,3,5],colors=['k'], linewidths=0.6)
cs2.clabel(fmt='%2d')
#cbar2 = plt.colorbar(cf1,pad=0.01);cbar2.set_label('Reg [g/kg]/[mm/hr]')
ax[1].set_ylim([1000,200]);ax[1].grid(linestyle='--')
ax[1].set_xticks(np.linspace(0,48,13));ax[1].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[1].set_xlabel('Lag time [hr]',fontsize=12)
ax[1].set_ylabel('Pressure [hPa]',fontsize=12)

cf3 = ax[2].contourf(range(49),lev,1e6*div_regress_orgmn.transpose(),levels=levs
                   ,cmap='PiYG',extend='both')
cs3 = ax[2].contour(range(49),lev,1e6*div_regress_orgmn.transpose()
            ,levels=[-5,-3,-1,1,3,5],colors=['k'], linewidths=0.6)
cs3.clabel(fmt='%2d')
#cbar3 = plt.colorbar(cf1,pad=0.01);cbar3.set_label('Reg [g/kg]/[mm/hr]')
ax[2].set_ylim([1000,200]);ax[2].grid(linestyle='--')
ax[2].set_xticks(np.linspace(0,48,13));ax[2].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[2].set_xlabel('Lag time [hr]',fontsize=12)
ax[2].set_ylabel('Pressure [hPa]',fontsize=12)

In [ ]:
cldfrac_bin = np.array([0,0.25,0.5,1]) # mm/day
T_bin = np.zeros((len(cldfrac_bin)-1,49,42)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    T_bin[n,:,:] = np.nanmean(T_all[idx,:,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(8,10))

levs = np.linspace(-5,5,21)
cf1 = ax[0].contourf(range(49),lev,(T_bin[0,:,:]-np.tile(T_bin[0,:,:].mean(axis=0),(49,1))).transpose()#,levels=levs
                   ,cmap='PiYG',extend='both')
ax[0].set_ylim([1000,200]);ax[0].grid(linestyle='--')
ax[0].set_xticks(np.linspace(0,48,13));ax[0].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[0].set_xlabel('Lag time [hr]',fontsize=12)
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)

cf2 = ax[1].contourf(range(49),lev,(T_bin[1,:,:]-np.tile(T_bin[1,:,:].mean(axis=0),(49,1))).transpose()#,levels=levs
                   ,cmap='PiYG',extend='both')
ax[1].set_ylim([1000,200]);ax[1].grid(linestyle='--')
ax[1].set_xticks(np.linspace(0,48,13));ax[1].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[1].set_xlabel('Lag time [hr]',fontsize=12)
ax[1].set_ylabel('Pressure [hPa]',fontsize=12)

cf3 = ax[2].contourf(range(49),lev,(T_bin[2,:,:]-np.tile(T_bin[2,:,:].mean(axis=0),(49,1))).transpose()#,levels=levs
                   ,cmap='PiYG',extend='both')
ax[2].set_ylim([1000,200]);ax[2].grid(linestyle='--')
ax[2].set_xticks(np.linspace(0,48,13));ax[2].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[2].set_xlabel('Lag time [hr]',fontsize=12)
ax[2].set_ylabel('Pressure [hPa]',fontsize=12)

In [ ]:
%%time
rh_scat = rh_all[np.where(cldfrac_all[:,24]<0.25)[0],:,:]
prec_scat = prec_all[np.where(cldfrac_all[:,24]<0.25)[0],:]
rh_regress_scat = np.copy(rh_scat)

for t in range(rh_regress_scat.shape[0]): # samples
    for k in range(rh_regress_scat.shape[2]): # vertical dims
        tmp = prec_scat[t,:];tmp2 = rh_scat[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            rh_regress_scat[t,:,k] = np.asarray(reg)
        else:
            rh_regress_scat[t,:,k] = np.nan
#---------
rh_mid = rh_all[np.where(np.logical_and(cldfrac_all[:,24]>0.25,cldfrac_all[:,24]<0.5))[0],:,:]
prec_mid = prec_all[np.where(np.logical_and(cldfrac_all[:,24]>0.25,cldfrac_all[:,24]<0.5))[0],:]
rh_regress_mid = np.copy(rh_mid)
for t in range(rh_regress_mid.shape[0]): # samples
    for k in range(rh_regress_mid.shape[2]): # vertical dims
        tmp = prec_mid[t,:];tmp2 = rh_mid[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            rh_regress_mid[t,:,k] = np.asarray(reg)
        else:
            rh_regress_mid[t,:,k] = np.nan
#----------
# lag-height regressed div and T based on precip
rh_org = rh_all[np.where(cldfrac_all[:,24]>0.5)[0],:,:]
prec_org = prec_all[np.where(cldfrac_all[:,24]>0.5)[0],:]
rh_regress_org = np.copy(div_org)

for t in range(rh_regress_org.shape[0]): # samples
    for k in range(rh_regress_org.shape[2]): # vertical dims
        tmp = prec_org[t,:];tmp2 = rh_org[t,:,k]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            rh_regress_org[t,:,k] = np.asarray(reg)
        else:
            rh_regress_org[t,:,k] = np.nan

In [ ]:
# composite significance 
stderr_scat = np.nanstd(rh_regress_scat,0)/rh_regress_scat.shape[0]
stderr_mid = np.nanstd(rh_regress_mid,0)/rh_regress_mid.shape[0]
stderr_org = np.nanstd(rh_regress_org,0)/rh_regress_org.shape[0]

rh_regress_scatmn = np.nanmean(rh_regress_scat,0)
#rh_regress_scatmn[abs(rh_regress_scatmn)<2*stderr_scat] = np.nan

rh_regress_midmn = np.nanmean(rh_regress_mid,0)
#rh_regress_midmn[abs(rh_regress_midmn)<2*stderr_mid] = np.nan

rh_regress_orgmn = np.nanmean(rh_regress_org,0)
#rh_regress_orgmn[abs(rh_regress_orgmn)<2*stderr_org] = np.nan

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(8,10))

levs = np.linspace(-12,12,25)
cf1 = ax[0].contourf(range(49),lev,rh_regress_scatmn.transpose(),levels=levs
                   ,cmap='BrBG',extend='both')
cs1 = ax[0].contour(range(49),lev,rh_regress_scatmn.transpose()
            ,levels=[],colors=['k'], linewidths=0.6)
cs1.clabel(fmt='%2d')
#cbar1 = plt.colorbar(cf1,pad=0.01);cbar1.set_label('Reg [g/kg]/[mm/hr]')
ax[0].set_ylim([1000,200]);ax[0].grid(linestyle='--')
ax[0].set_xticks(np.linspace(0,48,13));ax[0].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[0].set_xlabel('Lag time [hr]',fontsize=12)
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)

cf2 = ax[1].contourf(range(49),lev,rh_regress_midmn.transpose(),levels=levs
                   ,cmap='BrBG',extend='both')
cs2 = ax[1].contour(range(49),lev,rh_regress_midmn.transpose()
            ,levels=[],colors=['k'], linewidths=0.6)
cs2.clabel(fmt='%2d')
#cbar2 = plt.colorbar(cf1,pad=0.01);cbar2.set_label('Reg [g/kg]/[mm/hr]')
ax[1].set_ylim([1000,200]);ax[1].grid(linestyle='--')
ax[1].set_xticks(np.linspace(0,48,13));ax[1].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[1].set_xlabel('Lag time [hr]',fontsize=12)
ax[1].set_ylabel('Pressure [hPa]',fontsize=12)

cf3 = ax[2].contourf(range(49),lev,rh_regress_orgmn.transpose(),levels=levs
                   ,cmap='BrBG',extend='both')
cs3 = ax[2].contour(range(49),lev,rh_regress_orgmn.transpose()
            ,levels=[],colors=['k'], linewidths=0.6)
cs3.clabel(fmt='%2d')
#cbar3 = plt.colorbar(cf1,pad=0.01);cbar3.set_label('Reg [g/kg]/[mm/hr]')
ax[2].set_ylim([1000,200]);ax[2].grid(linestyle='--')
ax[2].set_xticks(np.linspace(0,48,13));ax[2].set_xticklabels(np.linspace(-72,72,13,dtype=int))
ax[2].set_xlabel('Lag time [hr]',fontsize=12)
ax[2].set_ylabel('Pressure [hPa]',fontsize=12)



### case view

In [ ]:
def caseview(time_event,lat_event,lon_event):

    data_sub1 = subset_MERRA2(MERRA2_3d_dir+'TQUV_3hr'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub2 = subset_TRMM(TRMM_dir+'2014_2018'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub3 = subset_MERRA2(MERRA2_dir+'inst_2d_hourly'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub4 = subset_MERRA2(MERRA2_3d_dir+'omega_3hr'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub5 = subset_gridsat(Gridsat_dir+'2014_2018'
                          ,time_event,lat_event,lon_event,3,10,20)
    
    u_sm = data_sub1.U
    v_sm = data_sub1.V
    T_sm = data_sub1.T
    q_sm = data_sub1.QV
    prec_sm = data_sub2.precipitation.resample(time='6H').nearest()
    cwv_sm = data_sub3.TQV.resample(time='6H').nearest()
    omega_sm = data_sub4.OMEGA
    bt_sm = data_sub5.irwin_cdr.resample(time='6H').nearest()

    u_950sm = u_sm.sel(lev=950).resample(time='6H').nearest()
    v_950sm = v_sm.sel(lev=950).resample(time='6H').nearest()
    u_tar = u_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    v_tar = v_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    omega_tar = omega_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    prec_tar = prec_sm.sel(nlon=slice(lon_event-5,lon_event+5),nlat=slice(lat_event-5,lat_event+5))
    bt_tar = bt_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    SCAIbt_sm,SCAI_p,Nbt_sm,N_prec = SCAI_index(prec_tar,bt_tar) # 6H data
    div_tar = np.gradient(u_tar,0.625*100*1e3,axis=3)+np.gradient(v_tar,0.5*100*1e3,axis=2) # divergence 3D
    
    p = T_sm.lev.values*units('mbar')
    T_tar = T_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    q_tar = q_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    # cape/cin calculation
    cape_sm = np.zeros(len(T_tar.time))
    cin_sm = np.copy(cape_sm)
    for t in range(len(u_tar.time)):
        T_tmp = np.nanmean(T_tar[t,:,:,:],axis=(1,2))
        q_tmp = np.nanmean(q_tar[t,:,:,:],axis=(1,2))
        T = (T_tmp-273.15)*units('degC')
        q = q_tmp*units('kg/kg')
        Td = mpcalc.dewpoint(mpcalc.vapor_pressure(p,q))
        
        try:
            Tp = mpcalc.parcel_profile(p,T[0],Td[0]).to('degC')                    
            el_pressure,el_temperature = mpcalc.el(p,T,Td) # equilibrium level
            el_idx = np.argmin(np.abs(p.magnitude - el_pressure.magnitude))                
            ELps = [el_pressure.magnitude] # Initialize an array of EL pressures for detrainment profile
            [CAPE,CIN] = mpcalc.cape_cin(p[:el_idx],T[:el_idx],Td[:el_idx],Tp[:el_idx])
            cape_sm[t] = CAPE.magnitude
            cin_sm[t] = CIN.magnitude
        except:
            cape_sm[t] = np.nan
            cin_sm[t] = np.nan
    
    # Abar calculation
    tmp = bt_tar.values
    cldfrac_sm = [len(np.where(tmp[n,:,:]<240)[0])/(tmp.shape[1]*tmp.shape[2]) for n in range(tmp.shape[0])]
    Abar_sm = cldfrac_sm/Nbt_sm*1600
    
    return (u_950sm,v_950sm,omega_sm,prec_sm,cwv_sm,bt_sm,SCAIbt_sm,Nbt_sm,Abar_sm
            ,div_tar,cape_sm,cin_sm)

In [ ]:
org_num = np.where(cldfrac_all[:,24]>0.5)[0]
scat_num =  np.where(np.logical_and(cldfrac_all[:,24]>=0,cldfrac_all[:,24]<0.1))[0]
mid_num =  np.where(np.logical_and(cldfrac_all[:,24]>=0.25,cldfrac_all[:,24]<0.5))[0]

num=50
u_950sm,v_950sm,omega_sm,prec_sm,cwv_sm,bt_sm,SCAIbt_sm,Nbt_sm,Abar_sm,div_sm,cape_sm,cin_sm = caseview(time_event[num],lat_event[num],lon_event[num])

In [ ]:
fig,ax = plt.subplots(7,2,figsize=(12,15))
for n in range(7):
    #ax[n] = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
    ax[n,0].pcolormesh(cwv_sm.lon,cwv_sm.lat,cwv_sm[4*n,:,:])
    ax[n,0].contour(cwv_sm.lon,cwv_sm.lat,cwv_sm[4*n,:,:],levels=[48],colors=['w'])
    ax[n,0].quiver(u_950sm.lon[::2],u_950sm.lat[::2],u_950sm[4*n,::2,::2],v_950sm[4*n,::2,::2])      
    rect1 = patches.Rectangle((lon_event[num]-5,lat_event[num]-5),10,10,linewidth=2,
                              edgecolor='r',facecolor='None')
    ax[n,0].add_patch(rect1)
    if n ==0:
        ax[n,0].set_title(time_event[num])
        
    ax[n,1].contour(prec_sm.nlon,prec_sm.nlat,prec_sm[4*n,:,:].transpose(),
                   levels=[1,5,10,30,50],linewidths=1,colors=['m'])
    tmp = bt_sm[4*n,:,:].values; #tmp[tmp>240] = np.nan
    ax[n,1].pcolormesh(bt_sm.lon,bt_sm.lat,tmp,cmap='binary_r') 
    rect2 = patches.Rectangle((lon_event[num]-5,lat_event[num]-5),10,10,linewidth=2,
                              edgecolor='r',facecolor='None')
    ax[n,1].add_patch(rect2)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+7.5,'N = '+str(int(Nbt_sm[n])),color='g',fontsize=11)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+5.5,'SCAI = '+str(round(SCAIbt_sm[n],1)),color='g',fontsize=11)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+3.5,'Abar = '+str(round(Abar_sm[n],1)),color='g',fontsize=11)
    if n ==0:
        ax[n,1].set_title(time_event[num])

### relationship between CRH/CWV and precipitation during events

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
crhcom_bin = np.zeros((len(cldfrac_bin)-1,49))
preccom_crh_bin = np.copy(crhcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>= cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    crhcom_bin[n,:] = np.nanmean(crh_all[idx,:],axis=0)
    preccom_crh_bin[n,:] = np.nanmean(prec_all[idx,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cwvcom_bin = np.zeros((len(cldfrac_bin)-1,49))
preccom_cwv_bin = np.copy(cwvcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>= cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwvcom_bin[n,:] = np.nanmean(cwv_all[idx,:],axis=0)
    preccom_cwv_bin[n,:] = np.nanmean(prec_all[idx,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
# SSMI/daily 2018 P-RH relation 
pr=6.89e-5 # mm/day
ad=14.72   

crh_range = np.linspace(0.3,0.9,50,dtype='float32')
P_est_R2018_all = pr*np.exp(ad*crh_range)/24
P_est_B2004_all = np.exp(11.4*(crh_range-0.522))/24

In [ ]:
cwv_array = np.linspace(40,65,25)
crh_array = np.linspace(0.5,0.9,25)
prec_cwv = np.zeros(len(cwv_array)-1)
prec_crh = np.zeros(len(crh_array)-1)

for i in range(len(cwv_array)-1):
    idx = np.where(np.logical_and(cwv_all.ravel()>=cwv_array[i],cwv_all.ravel()<cwv_array[i+1]))[0]
    prec_cwv[i] = np.nanmean(prec_all.ravel()[idx])
    
for i in range(len(crh_array)-1):
    idx = np.where(np.logical_and(crh_all.ravel()>=crh_array[i],crh_all.ravel()<crh_array[i+1]))[0]
    prec_crh[i] = np.nanmean(prec_all.ravel()[idx])

In [ ]:
crh_samp = crh_all.ravel()
prec_samp = prec_all.ravel()

crh_re = crh_samp[np.where(~np.isnan(crh_samp))[0]]
prec_re = prec_samp[np.where(~np.isnan(crh_samp))[0]]
# remove non-precip 
prec_re2 = prec_re[np.where(prec_re>0)[0]]
crh_re2 = crh_re[np.where(prec_re>0)[0]]

c = np.polyfit(crh_re2,np.log(prec_re2),1)

prec_clim = np.exp(c[0]*crh_range+c[1])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,6))

ax[0].plot(crhcom_bin[0,:],preccom_crh_bin[0,:],'-om',markerfacecolor='None')
ax[0].plot(crhcom_bin[1,:],preccom_crh_bin[1,:],'-og',markerfacecolor='None')
ax[0].plot(crhcom_bin[2,:],preccom_crh_bin[2,:],'-ob',markerfacecolor='None')
ax[0].plot(crhcom_bin[3,:],preccom_crh_bin[3,:],'-oy',markerfacecolor='None')
ax[0].plot(crh_array[:-1],prec_crh,'-k',linewidth=2.5)
ax[0].set_xlabel('Column relative humidity',fontsize=14)
ax[0].set_ylabel('Precipitation [mm/hr]',fontsize=14)
ax[0].set_xlim([0.55,0.85]);ax[0].set_ylim([-0.06,2.2])
ax[0].legend(['0-0.1','0.1-0.25','0.25-0.5','>0.5','CLIM'])
ax[0].grid(linestyle='--')
ax[0].tick_params(labelsize=13)

ax[1].plot(cwvcom_bin[0,:],preccom_cwv_bin[0,:],'-om',markerfacecolor='None')
ax[1].plot(cwvcom_bin[1,:],preccom_cwv_bin[1,:],'-og',markerfacecolor='None')
ax[1].plot(cwvcom_bin[2,:],preccom_cwv_bin[2,:],'-ob',markerfacecolor='None')
ax[1].plot(cwvcom_bin[3,:],preccom_cwv_bin[3,:],'-oy',markerfacecolor='None')
ax[1].plot(cwv_array[:-1],prec_cwv,'-k',linewidth=2.5)
ax[1].set_xlabel('Column water vapor [mm]',fontsize=14)
ax[1].set_ylabel('Precipitation [mm/hr]',fontsize=14)
ax[1].set_xlim([42,60]);plt.ylim([-0.06,2.2])
ax[1].legend(['0-0.1','0.1-0.25','0.25-0.5','>0.5','CLIM'])
ax[1].grid(linestyle='--')
ax[1].tick_params(labelsize=13)



In [ ]:
# linear interp to same size as cwvten_dyn
crh_samp = np.zeros((crh_all.shape[0],48))
prec_samp = np.zeros((crh_all.shape[0],48))
xp=np.linspace(1,49,49)
x=np.linspace(1.5,48.5,48)

for n in range(crh_all.shape[0]):
    crh_samp[n,:] = np.interp(x,xp,crh_all[n,:])
    prec_samp[n,:] = np.interp(x,xp,prec_all[n,:])
    
plt.scatter(crh_samp.ravel()/100,prec_samp.ravel(),s=10,c=cwvten_dyn_all.ravel(),
           cmap='RdBu',vmax=5e-4,vmin=-5e-4)
plt.colorbar()

In [ ]:
idx = np.where(np.logical_and(cldfrac_all[:,24]>= 0.5,cldfrac_all[:,24]< 10))[0]
crhtest = crh_all[idx,:]
prectest = prec_all[idx,:]

In [ ]:
cldfrac_bin = np.array([0,0.25,0.5,1]) # mm/day
crhcom_bin = np.zeros((len(cldfrac_bin)-1,49))
crhse_bin = np.copy(crhcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    crhcom_bin[n,:] = np.nanmean(crh_all[idx],axis=0)
    crhse_bin[n,:] = np.nanstd(crh_all[idx],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cwvcom_bin = np.zeros((len(cldfrac_bin)-1,49))
cwvse_bin = np.copy(cwvcom_bin)
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwvcom_bin[n,:] = np.nanmean(cwv_all[idx],axis=0)
    cwvse_bin[n,:] = np.nanstd(cwv_all[idx],axis=0)/len(idx)**0.5
    num_bin[n] = len(idx)

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cwvten_dyn_bin = np.zeros((len(cldfrac_bin)-1,48)) # m/s
cwvten_phy_bin = np.copy(cwvten_dyn_bin)
cwvten_ana_bin = np.copy(cwvten_dyn_bin)
cwvten_tot_bin = np.copy(cwvten_dyn_bin)

num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwvten_dyn_bin[n,:] = np.nanmean(cwvten_dyn_all[idx,:],axis=0)
    cwvten_phy_bin[n,:] = np.nanmean(cwvten_phy_all[idx,:],axis=0)
    cwvten_ana_bin[n,:] = np.nanmean(cwvten_ana_all[idx,:],axis=0)
    cwvten_tot_bin[n,:] = np.nanmean(cwvten_tot_all[idx,:],axis=0)

    num_bin[n] = len(idx)

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
fig = plt.figure(figsize=(6,6))

cwvten_all_bin = cwvten_dyn_bin + cwvten_phy_bin + cwvten_ana_bin

plt.plot(cwvcom_bin[0,:-1]/100,86400*cwvten_tot_bin[0,:],'-r',zorder=1,linewidth=1)
plt.scatter(cwvcom_bin[0,:-1]/100,86400*cwvten_tot_bin[0,:],c=range(-24,24),s=80
            ,cmap=discrete_cmap(6, 'jet'),zorder=2,edgecolor='grey')

plt.plot(cwvcom_bin[1,:-1]/100,86400*cwvten_tot_bin[1,:],'-m',zorder=1,linewidth=1)
plt.scatter(cwvcom_bin[1,:-1]/100,86400*cwvten_tot_bin[1,:],c=range(-24,24),s=80
            ,cmap=discrete_cmap(6, 'jet'),zorder=2,edgecolor='grey')

#plt.plot(cwvcom_bin[2,:-1]/100,86400*cwvten_tot_bin[2,:],'-g',zorder=1,linewidth=1)
#plt.scatter(cwvcom_bin[2,:-1]/100,86400*cwvten_tot_bin[2,:],c=range(-24,24),s=80
#            ,cmap=discrete_cmap(6, 'jet'),zorder=2,edgecolor='grey')

plt.plot(cwvcom_bin[3,:-1]/100,86400*cwvten_tot_bin[3,:],'-g',zorder=1,linewidth=1)
plt.scatter(cwvcom_bin[3,:-1]/100,86400*cwvten_tot_bin[3,:],c=range(-24,24),s=80
            ,cmap=discrete_cmap(6, 'jet'),zorder=2,edgecolor='grey')

#plt.plot(crhcom_bin[4,:-1]/100,86400*cwvten_tot_bin[4,:],'-g',zorder=1,linewidth=1)
#plt.scatter(crhcom_bin[4,:-1]/100,86400*cwvten_tot_bin[4,:],c=range(1,49),s=80,cmap='PiYG',zorder=2
#           ,edgecolor='grey')
plt.plot(np.linspace(0.3,0.8,51),np.zeros(51),'-.k')
plt.xlim([0.4,0.65])
plt.colorbar()

In [ ]:
### moisture budeget equation in MERRA2
plt.plot(cwvten_tot_bin[0,:]*3600*24,'k')
plt.plot(cwvten_tot_bin[1,:]*3600*24,'r')
plt.plot(cwvten_tot_bin[2,:]*3600*24,'g')
plt.plot(cwvten_tot_bin[3,:]*3600*24,'b')

In [ ]:
plt.plot(cwvten_dyn_bin[0,:]*3600*24,'k')
plt.plot(cwvten_dyn_bin[1,:]*3600*24,'r')
plt.plot(cwvten_dyn_bin[2,:]*3600*24,'g')
plt.plot(cwvten_dyn_bin[3,:]*3600*24,'b')
plt.ylim([-15,15])

In [ ]:
plt.plot(cwvten_phy_bin[0,:]*3600*24,'k')
plt.plot(cwvten_phy_bin[1,:]*3600*24,'r')
plt.plot(cwvten_phy_bin[2,:]*3600*24,'g')
plt.plot(cwvten_phy_bin[3,:]*3600*24,'b')
plt.ylim([-15,15])

In [ ]:
plt.plot(cwvten_ana_bin[0,:]*3600*24,'k')
plt.plot(cwvten_ana_bin[1,:]*3600*24,'--r')
plt.plot(cwvten_ana_bin[2,:]*3600*24,'--g')
plt.plot(cwvten_ana_bin[3,:]*3600*24,'--b')
plt.ylim([-15,15])

In [ ]:
#---------------------------------- end -------------------------

In [ ]:
def vars_sm(data_xr,time_center,lat_center,lon_center,t_window,box_deg=10):
    tmp = data_xr.sel(time=slice(time_center-timedelta(hours=t_window),time_center+timedelta(hours=t_window)),
                                lat = slice(lat_center-box_deg/2,lat_center+box_deg/2),
                                lon = slice(lon_center-box_deg/2,lon_center+box_deg/2))
    return tmp

In [ ]:
def match_file_MERRA2(file_path,time_event):
    date_str = datetime.strftime(time_event,format='%Y%m%d')
    os.chdir(file_path)
    file = glob('*'+date_str+'*')[0]
    #print(file)
    data = xr.open_dataset(file)
    return data

In [ ]:
def RH_event_lev(time_event,lat_event,lon_event,lev=slice(1000,850)):
    
    rh_mean = np.zeros(len(time_event))

    for t in range(len(time_event)):
    # return RH_mean at different layers 
        time = time_event[t]
        lon = lon_event[t]
        lat = lat_event[t]
        # load data file to avoid crazily long waiting...
        data = match_file_MERRA2(MERRA2_3d_dir+'TQUV_3hr/',time)
        data = data.sel(time=time,lat=slice(lat-5,lat+5),lon=slice(lon-5,lon+5),lev=lev)
        #print(data)
        T_xr = data.T; q_xr = data.QV; 
        plev = T_xr.lev
        es = 6.1094*np.exp(17.625*(T_xr-273)/(T_xr-273+243.04))
        p = (q_xr/q_xr)*plev
        qs = es/(p-es)*0.622
        rh_xr = q_xr/qs*100 # relative humidity [%] 
        
        rh_mean[t] = rh_xr.mean().values
        del data
    return rh_mean

In [ ]:
%%time
rh_mean_low = RH_event_lev(time_event,lat_event,lon_event,lev=slice(1000,850))
rh_mean_mid = RH_event_lev(time_event,lat_event,lon_event,lev=slice(700,550))
rh_mean_high = RH_event_lev(time_event,lat_event,lon_event,lev=slice(400,250))

In [ ]:
def CFAD_analysis(var1,var2,var3,bin1,bin2):

    bin_cell = np.zeros((len(bin1)-1,len(bin2)-1))
    num_cell = bin_cell.copy()

    for i in range(len(bin1)-1):
        for j in range(len(bin2)-1):
            idx = np.where(np.logical_and(np.logical_and(var1>=bin1[i],var1<bin1[i+1])
                                     ,np.logical_and(var2>=bin2[j],var2<bin2[j+1])))[0]
            bin_cell[i,j] = np.nanmean(var3[idx])
            num_cell[i,j] = len(idx)    
    return (bin_cell, num_cell)

In [ ]:
bin1 = np.linspace(20,100,17)
bin2 = np.linspace(0,60,13)

bin_lowc, num_lowc = CFAD_analysis(rh_mean_low,cu_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_lowd, num_lowd = CFAD_analysis(rh_mean_low,dc_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_lows, num_lows = CFAD_analysis(rh_mean_low,st_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))

bin_midc, num_midc = CFAD_analysis(rh_mean_mid,cu_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_midd, num_midd = CFAD_analysis(rh_mean_mid,dc_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_mids, num_mids = CFAD_analysis(rh_mean_mid,st_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))

bin_highc, num_highc = CFAD_analysis(rh_mean_high,cu_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_highd, num_highd = CFAD_analysis(rh_mean_high,dc_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))
bin_highs, num_highs = CFAD_analysis(rh_mean_high,st_event,prec_event,
                                   np.linspace(20,100,17),np.linspace(0,60,13))

In [ ]:
fig = plt.figure(figsize=(12,4))

ax1 = plt.subplot(1,3,1)
pc1 = ax1.pcolormesh(bin1,bin2,bin_lowc.transpose(),cmap='jet',vmax=60)
#cbar1 = plt.colorbar(pc1,shrink=0.8)
#cbar1.set_label('precipitation [mm/day]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_lowc.transpose()/np.sum(num_lowc),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('<RH>$_{low}$ [%]',fontsize=12);ax1.set_xlim([15,100])
ax1.set_ylabel('cloud amount [%]',fontsize=12);ax1.set_ylim([-4,60])
ax1.set_title('Precip, <RH>$_{low}$ and Cu_amt')

ax2 = plt.subplot(1,3,2)
pc2 = ax2.pcolormesh(bin1,bin2,bin_lowd.transpose(),cmap='jet',vmax=60)
#cbar2 = plt.colorbar(pc2,shrink=0.8)
#cbar2.set_label('precipitation [mm/day]')
cf2 = ax2.contour(bin1[:-1],bin2[:-1],num_lowd.transpose()/np.sum(num_lowd),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax2.clabel(cf2,[0.005,0.01,0.03,0.05],fontsize=8)
ax2.set_xlabel('<RH>$_{low}$ [%]',fontsize=12);ax2.set_xlim([15,100])
ax2.set_ylabel('cloud amount [%]',fontsize=12);ax2.set_ylim([-4,60])
ax2.set_title('Precip, <RH>$_{low}$ and Dc_amt')

ax3 = plt.subplot(1,3,3)
pc3 = ax3.pcolormesh(bin1,bin2,bin_lows.transpose(),cmap='jet',vmax=60)
cbar3 = plt.colorbar(pc3,shrink=0.8)
cbar3.set_label('precipitation [mm/day]')
cf3 = plt.contour(bin1[:-1],bin2[:-1],num_lows.transpose()/np.sum(num_lows),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax3.clabel(cf3,[0.005,0.01,0.03,0.05],fontsize=8)
ax3.set_xlabel('<RH>$_{low}$ [%]',fontsize=12);ax3.set_xlim([15,100])
ax3.set_ylabel('cloud amount [%]',fontsize=12);ax3.set_ylim([-4,60])
ax3.set_title('Precip, <RH>$_{low}$ and St_amt')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,4))

ax1 = plt.subplot(1,3,1)
pc1 = ax1.pcolormesh(bin1,bin2,bin_midc.transpose(),cmap='jet',vmax=60)
#cbar1 = plt.colorbar(pc1,shrink=0.8)
#cbar1.set_label('precipitation [mm/day]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_midc.transpose()/np.sum(num_midc),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('<RH>$_{mid}$ [%]',fontsize=12);ax1.set_xlim([15,100])
ax1.set_ylabel('cloud amount [%]',fontsize=12);ax1.set_ylim([-4,60])
ax1.set_title('Precip, <RH>$_{mid}$ and Cu_amt')

ax2 = plt.subplot(1,3,2)
pc2 = ax2.pcolormesh(bin1,bin2,bin_midd.transpose(),cmap='jet',vmax=60)
#cbar2 = plt.colorbar(pc2,shrink=0.8)
#cbar2.set_label('precipitation [mm/day]')
cf2 = ax2.contour(bin1[:-1],bin2[:-1],num_midd.transpose()/np.sum(num_midd),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax2.clabel(cf2,[0.005,0.01,0.03,0.05],fontsize=8)
ax2.set_xlabel('<RH>$_{mid}$ [%]',fontsize=12);ax2.set_xlim([15,100])
ax2.set_ylabel('cloud amount [%]',fontsize=12);ax2.set_ylim([-4,60])
ax2.set_title('Precip, <RH>$_{mid}$ and Dc_amt')

ax3 = plt.subplot(1,3,3)
pc3 = ax3.pcolormesh(bin1,bin2,bin_mids.transpose(),cmap='jet',vmax=60)
cbar3 = plt.colorbar(pc3,shrink=0.8)
cbar3.set_label('precipitation [mm/day]')
cf3 = plt.contour(bin1[:-1],bin2[:-1],num_mids.transpose()/np.sum(num_mids),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax3.clabel(cf3,[0.005,0.01,0.03,0.05],fontsize=8)
ax3.set_xlabel('<RH>$_{mid}$ [%]',fontsize=12);ax3.set_xlim([15,100])
ax3.set_ylabel('cloud amount [%]',fontsize=12);ax3.set_ylim([-4,60])
ax3.set_title('Precip, <RH>$_{mid}$ and St_amt')

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,4))

ax1 = plt.subplot(1,3,1)
pc1 = ax1.pcolormesh(bin1,bin2,bin_highc.transpose(),cmap='jet',vmax=60)
#cbar1 = plt.colorbar(pc1,shrink=0.8)
#cbar1.set_label('precipitation [mm/day]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_highc.transpose()/np.sum(num_highc),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('<RH>$_{high}$ [%]',fontsize=12);ax1.set_xlim([15,100])
ax1.set_ylabel('cloud amount [%]',fontsize=12);ax1.set_ylim([-4,60])
ax1.set_title('Precip, <RH>$_{high}$ and Cu_amt')

ax2 = plt.subplot(1,3,2)
pc2 = ax2.pcolormesh(bin1,bin2,bin_highd.transpose(),cmap='jet',vmax=60)
#cbar2 = plt.colorbar(pc2,shrink=0.8)
#cbar2.set_label('precipitation [mm/day]')
cf2 = ax2.contour(bin1[:-1],bin2[:-1],num_highd.transpose()/np.sum(num_highd),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax2.clabel(cf2,[0.005,0.01,0.03,0.05],fontsize=8)
ax2.set_xlabel('<RH>$_{high}$ [%]',fontsize=12);ax2.set_xlim([15,100])
ax2.set_ylabel('cloud amount [%]',fontsize=12);ax2.set_ylim([-4,60])
ax2.set_title('Precip, <RH>$_{high}$ and Dc_amt')

ax3 = plt.subplot(1,3,3)
pc3 = ax3.pcolormesh(bin1,bin2,bin_highs.transpose(),cmap='jet',vmax=60)
cbar3 = plt.colorbar(pc3,shrink=0.8)
cbar3.set_label('precipitation [mm/day]')
cf3 = plt.contour(bin1[:-1],bin2[:-1],num_highs.transpose()/np.sum(num_highs),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax3.clabel(cf3,[0.005,0.01,0.03,0.05],fontsize=8)
ax3.set_xlabel('<RH>$_{high}$ [%]',fontsize=12);ax3.set_xlim([15,100])
ax3.set_ylabel('cloud amount [%]',fontsize=12);ax3.set_ylim([-4,60])
ax3.set_title('Precip, <RH>$_{high}$ and St_amt')

plt.tight_layout()
plt.show()

In [ ]:
# individual Aeaa 
bin1 = np.linspace(0,16,17)
bin2 = np.linspace(0,35,17)
Abarp_event = np.sqrt(precfrac_event/Nprec_event*1600) # pixels
Abarbt_event = np.sqrt(cldfrac_event/NpsT_event*1600) # pixels
bin_cell, num_cell = CFAD_analysis(SCAIbt_event,Abarbt_event,prec_event,
                                   bin1,bin2)

In [ ]:
fig = plt.figure(figsize=(5,4))

ax1 = plt.subplot()
pc1 = ax1.pcolormesh(bin1,bin2,bin_cell.transpose(),cmap='jet',vmax=60)
cbar1 = plt.colorbar(pc1,shrink=0.8)
cbar1.set_label('precipitation [mm/day]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_cell.transpose()/np.sum(num_cell),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('SCAIP',fontsize=12);ax1.set_xlim([0,10])
ax1.set_ylabel('$\sqrt{Area}$',fontsize=12);ax1.set_ylim([0,25])

In [ ]:
# low-level wind shear
bin1 = np.linspace(0,35,17)
bin2 = np.linspace(0,12,17)
bin_cell, num_cell = CFAD_analysis(Abarbt_event,llws_event,prec_event,
                                   bin1,bin2)

In [ ]:
fig = plt.figure(figsize=(5,4))

ax1 = plt.subplot()
pc1 = ax1.pcolormesh(bin1,bin2,bin_cell.transpose(),cmap='jet')
cbar1 = plt.colorbar(pc1,shrink=0.8)
cbar1.set_label('prec [mm/hr]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_cell.transpose()/np.sum(num_cell),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('Abar_bt',fontsize=12);ax1.set_xlim([0,20])
ax1.set_ylabel('LLWS',fontsize=12);ax1.set_ylim([0,8])

In [ ]:
# low-level wind shear
bin1 = np.linspace(0,16,17)
bin2 = np.linspace(0,35,17)
bin_cell, num_cell = CFAD_analysis(SCAIbt_event,Abarbt_event,bt_event,
                                   bin1,bin2)

In [ ]:
fig = plt.figure(figsize=(5,4))

ax1 = plt.subplot()
pc1 = ax1.pcolormesh(bin1,bin2,bin_cell.transpose(),cmap='jet')
cbar1 = plt.colorbar(pc1,shrink=0.8)
cbar1.set_label('BT [K]')
cf1 = ax1.contour(bin1[:-1],bin2[:-1],num_cell.transpose()/np.sum(num_cell),[0.005,0.01,0.03,0.05]
           ,colors=['k'],linewidths=0.7)
ax1.clabel(cf1,[0.005,0.01,0.03,0.05],fontsize=8)
ax1.set_xlabel('SCAIP',fontsize=12);ax1.set_xlim([0,10])
ax1.set_ylabel('$\sqrt{Area}$',fontsize=12);ax1.set_ylim([0,25])

In [ ]:
from skimage.filters import sobel

In [ ]:
a = np.random.randn(20,20)+10
a.shape

In [ ]:
cwv_xr[0,:,:].compute().values

In [ ]:
from scipy.ndimage import sobel as sobel2

In [ ]:
sobel2(cwv_xr[0,:,:].compute().values)

In [ ]:
# individual Aeaa 
t_org = np.where(np.logical_and(SCAIbt_event < 2, 
                                  Abarbt_event > 6,))[0] # org
t_scat = np.where(np.logical_and(SCAIbt_event > 3,
                                   Abarbt_event < 4))[0] # scattered

In [ ]:
print(len(t_org),len(t_scat))

In [ ]:
num = t_org[20]

prec_tmp = prec_xr.sel(time=time_event[num],lat=slice(lat_event[num]-5,lat_event[num]+5),
                  lon=slice(lon_event[num]-5,lon_event[num]+5))
bt_tmp = bt_xr.sel(time=time_event[num],lat=slice(lat_event[num]-5,lat_event[num]+5),
                  lon=slice(lon_event[num]-5,lon_event[num]+5))
cwv_tmp = cwv_xr.sel(time=time_event[num],lat=slice(lat_event[num]-5,lat_event[num]+5),
                  lon=slice(lon_event[num]-5,lon_event[num]+5))
#cwvera_tmp = cwv_era_xr.sel(time=time_event[num],latitude=slice(lat_event[num]+5,lat_event[num]-5),
#                  longitude=slice(lon_event[num]-5,lon_event[num]+5))
omega_tmp = Omega_xr.sel(time=time_event[num],lat=slice(lat_event[num]-5,lat_event[num]+5),
                  lon=slice(lon_event[num]-5,lon_event[num]+5),lev=500)

plt.figure(figsize=(7,5))
plt.contourf(cwv_tmp.lon,cwv_tmp.lat,cwv_tmp,[30,35,40,45,50,55,60,65,70])
plt.colorbar()
plt.contour(prec_tmp.lon,prec_tmp.lat,prec_tmp,[1])
#plt.contour(omega_tmp.lon,omega_tmp.lat,omega_tmp,[-2e-1,-1e-1,1e-1,2e-1],colors=['r'],)
plt.contour(bt_tmp.lon,bt_tmp.lat,bt_tmp,[180,220,240],colors=['r'],)

print('time:',time_event[num])
print('SCAIbt:', SCAIbt_event[num])
print('precfrac: ',precfrac_event[num])
print('N_bt:',NpsT_event[num])
print('cwv_dmean:',cwv_event[num])
print('prec_dmean',prec_event[num])
print('A_bar:',np.sqrt(precfrac_event[num]/Nprec_event[num]*1600))

In [ ]:
lat_bin = np.linspace(-10,10,5)
lon_bin = np.linspace(-175,180,72)
freq_bin = np.zeros((len(lon_bin),len(lat_bin)))
precamt_bin = freq_bin.copy()

fig = plt.figure(figsize=(14,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))

for i in range(len(lon_bin)):
    for j in range(len(lat_bin)):
        idx = np.where(np.logical_and(lon_event[t_org]==lon_bin[i],lat_event[t_org]==lat_bin[j]))[0]
        freq_bin[i,j]+=len(idx)
        if len(idx) > 0:
            precamt_bin[i,j]+=prec_event[t_org[idx]].sum()

lat_plt = [-10,5,0,5,10]
plt.pcolor(lon_bin,lat_bin[:-1],freq_bin[:,1:].transpose(),cmap='jet')
cbar = plt.colorbar(fraction=0.005,pad=0.02)
cbar.set_label('freqency')
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND
gl = ax.gridlines(draw_labels=True,linestyle='--')
gl.xlabels_top = gl.ylabels_right = gl.ylabels_left = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax.set_ylim([-20,20])

plt.show()

In [ ]:
lat_bin = np.linspace(-10,10,5)
lon_bin = np.linspace(-175,180,72)
freq_bin = np.zeros((len(lon_bin),len(lat_bin)))
precamt_bin = freq_bin.copy()

fig = plt.figure(figsize=(14,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))

for i in range(len(lon_bin)):
    for j in range(len(lat_bin)):
        idx = np.where(np.logical_and(lon_event[t_scat]==lon_bin[i],lat_event[t_scat]==lat_bin[j]))[0]
        freq_bin[i,j]+=len(idx)
        if len(idx) > 0:
            precamt_bin[i,j]+=prec_event[t_scat[idx]].sum()

lat_plt = [-10,5,0,5,10]
plt.pcolor(lon_bin,lat_bin[:-1],freq_bin[:,1:].transpose(),cmap='jet')
cbar = plt.colorbar(fraction=0.005,pad=0.02)
cbar.set_label('freqency')
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND
gl = ax.gridlines(draw_labels=True,linestyle='--')
gl.xlabels_top = gl.ylabels_right = gl.ylabels_left = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax.set_ylim([-20,20])

plt.show()

In [ ]:
for i,t in enumerate(t_scat):
    print(i)
    rh_sm = rh_xr.sel(time=slice(time_event[t]-timedelta(days=3),time_event[t]+timedelta(days=3)),
                    lat=slice(lat_event[t]-5,lat_event[t]+5),
                    lon=slice(lon_event[t]-5,lon_event[t]+5))

    tmp3 = np.nanmean(np.nanmean(rh_sm,axis=2),axis=2)
    rh_scat[i,:,:] = tmp3 - np.nanmean(tmp3,axis=0)